# TD Ameritrade API

Install modules

In [3]:
# import sys
# !{sys.executable} -m pip install

Load modules

In [4]:
from bakery.data.td import TDClient

In [153]:
import os
import requests
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from datetime import datetime, date, time, timedelta, timezone
import pendulum
from bakery.data.calendars import USHolidaysCalendar
from cassandra.cluster import Cluster

Set project directory and paths to credentials and tokens files

In [6]:
base_path = os.path.join(os.path.expanduser('~'), 'bakery')
creds_path = os.path.join(os.path.expanduser('~'), 'git/bakery/bakery/creds.json')

Get authentication object

In [242]:
td_client = TDClient(creds_path)
td_client.refresh_tokens()
td_auth = td_client.get_auth()

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


Get list of symbols from Cassandra table: bakery.symbols

In [8]:
cluster = Cluster()
session = cluster.connect()

rows = session.execute('select * from bakery.symbols')
first_row = rows.one()
columns = list(first_row._fields)
rows = [
    [row.symbol, row.country, row.industry, row.ipo_year, row.last_sale,
    row.market_cap, row.name, row.net_change, row.percent_change, row.sector, 
    row.volume, row.updated] for row in rows
]

symbols = pd.DataFrame(rows, columns=columns)

In [34]:
cluster = Cluster()
session = cluster.connect()

rows = session.execute('select symbol, market_cap from bakery.symbols')
rows = [[row.symbol, row.market_cap] for row in rows]
rows.sort(key=lambda x: x[1])
top100 = [row[0] for row in rows[-100:]]
top100

['AMT',
 'GE',
 'INTU',
 'PLD',
 'BLK',
 'ELV',
 'AMD',
 'AXP',
 'CVS',
 'PDD',
 'SPGI',
 'TD',
 'LMT',
 'SBUX',
 'BUD',
 'SNY',
 'HDB',
 'INTC',
 'GS',
 'RIO',
 'LOW',
 'BA',
 'UL',
 'DE',
 'IBM',
 'UNP',
 'CAT',
 'QCOM',
 'SAP',
 'RY',
 'T',
 'HSBC',
 'AMGN',
 'HON',
 'NFLX',
 'RTX',
 'CRM',
 'COP',
 'BMY',
 'MS',
 'SCHW',
 'PM',
 'UPS',
 'TTE',
 'ADBE',
 'TXN',
 'LIN',
 'WFC',
 'CMCSA',
 'NEE',
 'BHP',
 'VZ',
 'ACN',
 'DIS',
 'TMUS',
 'ABT',
 'TM',
 'NVS',
 'MCD',
 'DHR',
 'NKE',
 'CSCO',
 'SHEL',
 'COST',
 'AZN',
 'TMO',
 'GEN',
 'ORCL',
 'PEP',
 'AVGO',
 'ASML',
 'KO',
 'PFE',
 'ABBV',
 'BAC',
 'MRK',
 'BABA',
 'NVO',
 'META',
 'HD',
 'LLY',
 'CVX',
 'PG',
 'MA',
 'TSLA',
 'WMT',
 'NVDA',
 'JPM',
 'TSM',
 'JNJ',
 'UNH',
 'XOM',
 'V',
 'BRK/B',
 'BRK/A',
 'AMZN',
 'GOOGL',
 'GOOG',
 'MSFT',
 'AAPL']

In [35]:
len(top100)

100

## Price History

Get price history for one symbol

In [236]:
today = pendulum.today('UTC')
yesterday = today.subtract(days=1)
# yesterday_dt = today_dt.subtract(days=1)
# current_year = today_dt.year
# today_dow = today_dt.day_of_week

In [237]:
cal = USHolidaysCalendar()
holidays = cal.holidays(
    start=pendulum.datetime(yesterday.year, 1, 1, tz='UTC'),
    end = pendulum.datetime(yesterday.year, 12, 31, tz='UTC')
    )
yesterday_holiday = yesterday in holidays
yesterday_holiday

False

In [238]:
yesterday_weekend = yesterday.day_of_week in (0,6)
yesterday_weekend

False

In [241]:
yesterday_ts = yesterday.combine(yesterday.date(), pendulum.Time(6, 0, 0)).replace(tzinfo=pendulum.timezone('UTC')).int_timestamp * 1000
yesterday_ts

1674108000000

In [186]:
# if today_dt.day_of_week = 1:
#     ts = today_dt.subtract(days=3).add(hours=6).int_timestamp * 1000
# elif today_dt

# start_ts = pendulum.today('UTC').subtract(days=5).int_timestamp * 1000
# # end_ts = pendulum.today('UTC').subtract(days=1).add(hours=6).int_timestamp * 1000
# print(start_ts, end_ts)

In [248]:
params={
    # 'period': 1,
    'periodType': 'month',
    'frequencyType': 'daily',
    'frequency': 1,
    'endDate':yesterday_ts,
    'startDate':yesterday_ts,
    'needExtendedHoursData': 'false'
}

r = requests.get(
        f'https://api.tdameritrade.com/v1/marketdata/BRK.A/pricehistory',
        auth=td_auth,
        params=params
    )

In [249]:
r.json()['candles']

[{'open': 462497.5,
  'high': 465064.99,
  'low': 460233.0,
  'close': 461949.99,
  'volume': 3632,
  'datetime': 1674108000000}]

In [255]:
if r.json()['empty']:
    print('Candles emtpy')
else:
    print('Candles not empty')

Candles not empty


In [209]:
for candle in r.json()['candles']:
    dt = datetime.fromtimestamp(candle['datetime']/1000).strftime('%Y-%m-%d')
    values = (
        dt,
        int(candle['datetime']/1000),
        'MSFT',
        candle['open'],
        candle['high'],
        candle['low'],
        candle['close'],
        candle['volume']
    )
    print(values)

('2023-01-17', 1673965800, 'MSFT', 481590.0, 483992.7488, 480282.25, 482717.7488, 2696)
('2023-01-17', 1673965860, 'MSFT', 481717.5, 482729.0, 480260.0, 481810.0, 16)
('2023-01-17', 1673965920, 'MSFT', 480565.0, 481659.9988, 480280.0, 481400.0, 188)
('2023-01-17', 1673965980, 'MSFT', 481600.0, 481600.0, 480610.01, 481016.485, 17)
('2023-01-17', 1673966040, 'MSFT', 481129.96, 481249.96, 479695.0, 479944.98, 19)
('2023-01-17', 1673966100, 'MSFT', 480229.96, 480469.96, 479022.24, 479022.24, 9)
('2023-01-17', 1673966160, 'MSFT', 479824.9588, 480244.96, 478928.53, 479869.96, 25)
('2023-01-17', 1673966220, 'MSFT', 478938.0, 479719.9588, 478844.0, 479494.96, 19)
('2023-01-17', 1673966280, 'MSFT', 479524.9588, 479824.9588, 478775.0, 479764.9588, 15)
('2023-01-17', 1673966340, 'MSFT', 479674.9588, 479674.9588, 479219.98, 479629.9588, 17)
('2023-01-17', 1673966400, 'MSFT', 479314.985, 479884.9588, 478090.0, 478090.0, 20)
('2023-01-17', 1673966460, 'MSFT', 478684.9588, 479134.9588, 478090.01, 479

In [189]:
r.json()

{'error': 'The access token being passed has expired or is invalid.'}

Get price history for top 100 symbols by market cap

In [38]:
# Set request parameters
period_type = 'day'
period = 1
frequency_type = 'minute'
frequency = 1
end_date=today_ts,
start_date=yesterday_ts
# Set API rate limiting parameters
max_calls_per_minute = 100
num_calls = 0
start_time = time.time()
calls_per_minute = 0

for symbol in (pbar := tqdm(symbols_top100[:10])):
    # Update progress bar
    pbar.set_description(f'Symbol: {symbol}, Requests/Min: {calls_per_minute}')
    # Initialize empty response flag
    empty_response = False
    # Set request parameters
    if start_date:
        params={
            'frequencyType': frequency_type,
            'frequency': frequency,
            'endDate': end_date,
            'startDate': start_date
        }
    else: 
        params={
            'periodType': period_type,
            'period': period,
            'frequencyType': frequency_type,
            'frequency': frequency
        }
    # Get new access token if expired
    if time.time() > td_client.token['expires_at']:
        td_client = TDClient(creds_path)
        td_client.refresh_tokens()
        td_auth = td_client.get_auth()
    # Replace forward slashes with periods
    symbol = symbol.replace('/', '.')
    # Send get request
    r = requests.get(
            f'https://api.tdameritrade.com/v1/marketdata/{symbol}/pricehistory',
            auth=td_auth,
            params=params
        )
    # Update number of calls
    num_calls += 1
    # If response is not empty get candle values, otherwise set empty response flag to true
    try:
        candles = r.json()['candles']
    except (KeyError, ValueError):
        print(f'Symbol {symbol} returned empty response')
        empty_response = True
    
    # Write data to bakery.prices_daily Cassandra table
    if not empty_response:
        for candle in r.json()['candles']:
                # dt = datetime.fromtimestamp(candle['datetime']/1000).strftime('%Y-%m-%d')
            columns = (
                'dt' if frequency_type == 'daily' else 'dttm',
                'symbol', 'open_price', 'high_price', 'low_price', 'close_price', 'total_volume'
            )
            values = (
                int(candle['datetime'] / 1000) if frequency_type == 'daily' else candle['datetime'],
                symbol.replace('.', '/'), # Replace period with forward slash (original symbol)
                candle['open'],
                candle['high'],
                candle['low'],
                candle['close'],
                candle['volume']
            )
            # Get indices of null values so we only write to relevant columns
            null_idxs = [i for i, value in enumerate(values) if value == 'NaN']
            non_null_values = tuple([value for value in values if value != 'NaN'])
            non_null_columns = '(' + ', '.join([column for i, column in enumerate(columns) if i not in null_idxs]) + ')'
            # Create query
            query = f'''
                INSERT INTO bakery.prices_{frequency_type} {non_null_columns}
                VALUES {non_null_values}
            '''
            # Execute query
            # session.execute(query)
    # Update API rate limiting parameters
    elapsed_time = (time.time() - start_time) / 60
    calls_per_minute = num_calls / elapsed_time
    # If max calls / minute is exceeded sleep for one second
    if calls_per_minute > max_calls_per_minute:
        time.sleep(1)

Symbol: AAPL, Requests/Min: 0:   0%|          | 0/10 [00:00<?, ?it/s]

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


Symbol: UNH, Requests/Min: 99.57783563206829: 100%|██████████| 10/10 [00:06<00:00,  1.45it/s] 


In [39]:
query

"\n                INSERT INTO bakery.prices_minute (dttm, symbol, open_price, high_price, low_price, close_price, total_volume)\n                VALUES (1674069540000, 'UNH', 478.64, 478.7332, 478.605, 478.65, 3793)\n            "

## Equity Quotes

Get a quote for one symbol

In [13]:
symbol = 'AAPL'

params = {'symbol': symbol}

r = requests.get(
            'https://api.tdameritrade.com/v1/marketdata/quotes',
            auth=td_auth,
            params=params
        )

In [16]:
values = (
            'AAPL',
            r.json()['AAPL']['assetType'],
            r.json()['AAPL']['assetMainType'],
            r.json()['AAPL']['cusip'],
            r.json()['AAPL']['assetSubType'],
            r.json()['AAPL']['description'].replace("'", ""),
            r.json()['AAPL']['bidPrice'],
            r.json()['AAPL']['bidSize'],
            r.json()['AAPL']['bidId'],
            r.json()['AAPL']['askPrice'],
            r.json()['AAPL']['askSize'],
            r.json()['AAPL']['askId'],
            r.json()['AAPL']['lastPrice'],
            r.json()['AAPL']['lastSize'],
            r.json()['AAPL']['lastId'],
            r.json()['AAPL']['openPrice'],
            r.json()['AAPL']['highPrice'],
            r.json()['AAPL']['lowPrice'],
            r.json()['AAPL']['bidTick'],
            r.json()['AAPL']['closePrice'],
            r.json()['AAPL']['netChange'],
            r.json()['AAPL']['totalVolume'],
            r.json()['AAPL']['quoteTimeInLong'],
            r.json()['AAPL']['tradeTimeInLong'],
            r.json()['AAPL']['mark'],
            r.json()['AAPL']['exchange'],
            r.json()['AAPL']['exchangeName'],
            r.json()['AAPL']['marginable'],
            r.json()['AAPL']['shortable'],
            r.json()['AAPL']['volatility'],
            r.json()['AAPL']['digits'],
            r.json()['AAPL']['52WkHigh'],
            r.json()['AAPL']['52WkLow'],
            r.json()['AAPL']['nAV'],
            r.json()['AAPL']['peRatio'],
            r.json()['AAPL']['divAmount'],
            r.json()['AAPL']['divYield'],
            r.json()['AAPL']['divDate'],
            r.json()['AAPL']['securityStatus'],
            r.json()['AAPL']['regularMarketLastPrice'],
            r.json()['AAPL']['regularMarketLastSize'],
            r.json()['AAPL']['regularMarketNetChange'],
            r.json()['AAPL']['regularMarketTradeTimeInLong'],
            r.json()['AAPL']['netPercentChangeInDouble'],
            r.json()['AAPL']['markChangeInDouble'],
            r.json()['AAPL']['markPercentChangeInDouble'],
            r.json()['AAPL']['regularMarketPercentChangeInDouble'],
            r.json()['AAPL']['delayed'],
            r.json()['AAPL']['realtimeEntitled'],        
        )
values

('AAPL',
 'EQUITY',
 'EQUITY',
 '037833100',
 '',
 'Apple Inc. - Common Stock',
 136.66,
 600,
 'Q',
 136.67,
 200,
 'Q',
 136.6699,
 1000,
 'Q',
 136.815,
 138.61,
 136.55,
 ' ',
 135.94,
 0.7299,
 26029044,
 1674058351173,
 1674058350995,
 136.6699,
 'q',
 'NASD',
 True,
 True,
 0.0253,
 4,
 179.61,
 124.17,
 0.0,
 22.0842,
 0.92,
 0.68,
 '2022-11-04 00:00:00.000',
 'Normal',
 136.6699,
 10,
 0.7299,
 1674058350995,
 0.5369,
 0.7299,
 0.5369,
 0.5369,
 False,
 True)

Get price history for top 100 symbols by market cap

In [26]:
symbols_top100 = [symbol.replace('/', '.') for symbol in symbols_top100]
symbols_top100

['AAPL',
 'MSFT',
 'GOOG',
 'GOOGL',
 'AMZN',
 'BRK.A',
 'BRK.B',
 'V',
 'XOM',
 'UNH',
 'JNJ',
 'TSM',
 'JPM',
 'NVDA',
 'WMT',
 'TSLA',
 'MA',
 'PG',
 'CVX',
 'LLY',
 'HD',
 'META',
 'NVO',
 'BABA',
 'MRK',
 'BAC',
 'ABBV',
 'PFE',
 'KO',
 'ASML',
 'AVGO',
 'PEP',
 'ORCL',
 'GEN',
 'TMO',
 'AZN',
 'COST',
 'SHEL',
 'CSCO',
 'NKE',
 'DHR',
 'MCD',
 'NVS',
 'TM',
 'ABT',
 'TMUS',
 'DIS',
 'ACN',
 'VZ',
 'BHP',
 'NEE',
 'CMCSA',
 'WFC',
 'LIN',
 'TXN',
 'ADBE',
 'TTE',
 'UPS',
 'PM',
 'SCHW',
 'MS',
 'BMY',
 'COP',
 'CRM',
 'RTX',
 'NFLX',
 'HON',
 'AMGN',
 'HSBC',
 'T',
 'RY',
 'SAP',
 'QCOM',
 'CAT',
 'UNP',
 'IBM',
 'DE',
 'UL',
 'BA',
 'LOW',
 'RIO',
 'GS',
 'INTC',
 'HDB',
 'SNY',
 'BUD',
 'SBUX',
 'LMT',
 'TD',
 'SPGI',
 'PDD',
 'CVS',
 'AXP',
 'AMD',
 'ELV',
 'BLK',
 'PLD',
 'INTU',
 'GE',
 'AMT']

In [224]:


symbols_str = ','.join(symbols_top100)
    
params = {
        'symbol': symbols_str
    }

r = requests.get(
        'https://api.tdameritrade.com/v1/marketdata/quotes',
        auth=td_auth,
        params=params
    )

quotes = r.json()

In [225]:
symbols_str

'AAPL,MSFT,GOOG,GOOGL,AMZN,BRK.A,BRK.B,V,XOM,UNH,JNJ,TSM,JPM,NVDA,WMT,TSLA,MA,PG,CVX,LLY,HD,META,NVO,BABA,MRK,BAC,ABBV,PFE,KO,ASML,AVGO,PEP,ORCL,GEN,TMO,AZN,COST,SHEL,CSCO,NKE,DHR,MCD,NVS,TM,ABT,TMUS,DIS,ACN,VZ,BHP,NEE,CMCSA,WFC,LIN,TXN,ADBE,TTE,UPS,PM,SCHW,MS,BMY,COP,CRM,RTX,NFLX,HON,AMGN,HSBC,T,RY,SAP,QCOM,CAT,UNP,IBM,DE,UL,BA,LOW,RIO,GS,INTC,HDB,SNY,BUD,SBUX,LMT,TD,SPGI,PDD,CVS,AXP,AMD,ELV,BLK,PLD,INTU,GE,AMT'

In [227]:
quotes['MSFT']

{'assetType': 'EQUITY',
 'assetMainType': 'EQUITY',
 'cusip': '594918104',
 'assetSubType': '',
 'symbol': 'MSFT',
 'description': 'Microsoft Corporation - Common Stock',
 'bidPrice': 235.52,
 'bidSize': 200,
 'bidId': 'P',
 'askPrice': 235.58,
 'askSize': 100,
 'askId': 'Q',
 'lastPrice': 235.58,
 'lastSize': 0,
 'lastId': 'D',
 'openPrice': 241.565,
 'highPrice': 242.38,
 'lowPrice': 235.52,
 'bidTick': ' ',
 'closePrice': 240.35,
 'netChange': -4.77,
 'totalVolume': 29505189,
 'quoteTimeInLong': 1674079670158,
 'tradeTimeInLong': 1674079698099,
 'mark': 235.58,
 'exchange': 'q',
 'exchangeName': 'NASD',
 'marginable': True,
 'shortable': True,
 'volatility': 0.0164,
 'digits': 4,
 '52WkHigh': 315.95,
 '52WkLow': 213.431,
 'nAV': 0.0,
 'peRatio': 25.7779,
 'divAmount': 2.72,
 'divYield': 1.13,
 'divDate': '2023-02-15 00:00:00.000',
 'securityStatus': 'Normal',
 'regularMarketLastPrice': 235.81,
 'regularMarketLastSize': 37911,
 'regularMarketNetChange': -4.54,
 'regularMarketTradeTim

In [223]:
[quote for quote in r.json()]

['AAPL',
 'MSFT',
 'GOOG',
 'GOOGL',
 'AMZN',
 'BRK.A',
 'BRK.B',
 'V',
 'XOM',
 'UNH',
 'JNJ',
 'TSM',
 'JPM',
 'NVDA',
 'WMT',
 'TSLA',
 'MA',
 'PG',
 'CVX',
 'LLY',
 'HD',
 'META',
 'NVO',
 'BABA',
 'MRK',
 'BAC',
 'ABBV',
 'PFE',
 'KO',
 'ASML',
 'AVGO',
 'PEP',
 'ORCL',
 'GEN',
 'TMO',
 'AZN',
 'COST',
 'SHEL',
 'CSCO',
 'NKE',
 'DHR',
 'MCD',
 'NVS',
 'TM',
 'ABT',
 'TMUS',
 'DIS',
 'ACN',
 'VZ',
 'BHP',
 'NEE',
 'CMCSA',
 'WFC',
 'LIN',
 'TXN',
 'ADBE',
 'TTE',
 'UPS',
 'PM',
 'SCHW',
 'MS',
 'BMY',
 'COP',
 'CRM',
 'RTX',
 'NFLX',
 'HON',
 'AMGN',
 'HSBC',
 'T',
 'RY',
 'SAP',
 'QCOM',
 'CAT',
 'UNP',
 'IBM',
 'DE',
 'UL',
 'BA',
 'LOW',
 'RIO',
 'GS',
 'INTC',
 'HDB',
 'SNY',
 'BUD',
 'SBUX',
 'LMT',
 'TD',
 'SPGI',
 'PDD',
 'CVS',
 'AXP',
 'AMD',
 'ELV',
 'BLK',
 'PLD',
 'INTU',
 'GE',
 'AMT']

In [220]:
len(r.json())

100

In [222]:
r.json()['AAPL']

{'assetType': 'EQUITY',
 'assetMainType': 'EQUITY',
 'cusip': '037833100',
 'assetSubType': '',
 'symbol': 'AAPL',
 'description': 'Apple Inc. - Common Stock',
 'bidPrice': 134.96,
 'bidSize': 300,
 'bidId': 'P',
 'askPrice': 135.03,
 'askSize': 200,
 'askId': 'P',
 'lastPrice': 134.995,
 'lastSize': 0,
 'lastId': 'D',
 'openPrice': 136.815,
 'highPrice': 138.61,
 'lowPrice': 135.03,
 'bidTick': ' ',
 'closePrice': 135.94,
 'netChange': -0.945,
 'totalVolume': 69462744,
 'quoteTimeInLong': 1674079508600,
 'tradeTimeInLong': 1674079566808,
 'mark': 135.03,
 'exchange': 'q',
 'exchangeName': 'NASD',
 'marginable': True,
 'shortable': True,
 'volatility': 0.0175,
 'digits': 4,
 '52WkHigh': 179.61,
 '52WkLow': 124.17,
 'nAV': 0.0,
 'peRatio': 22.0842,
 'divAmount': 0.92,
 'divYield': 0.68,
 'divDate': '2022-11-04 00:00:00.000',
 'securityStatus': 'Normal',
 'regularMarketLastPrice': 135.21,
 'regularMarketLastSize': 1,
 'regularMarketNetChange': -0.73,
 'regularMarketTradeTimeInLong': 1674